In [ ]:
import pandas as pd

def generate_fico_rating_map(data: pd.DataFrame, n_buckets: int) -> pd.DataFrame:
    """
    Generates a FICO rating map by quantizing the FICO score into n_buckets
    using quantile-based binning (a general approach to minimize approximation error).

    The mapping is structured so that a lower rating (1) signifies a better credit score
    (higher FICO) and a lower default rate.

    Args:
        data (pd.DataFrame): DataFrame containing 'fico_score', 'default', and 'customer_id' columns.
        n_buckets (int): The desired number of rating buckets (e.g., 5 for R1-R5).

    Returns:
        pd.DataFrame: A table summarizing the rating map, including FICO range and default rate.
    """
    required_cols = ['fico_score', 'default', 'customer_id']
    if not all(col in data.columns for col in required_cols):
        raise ValueError(f"Input DataFrame must contain: {required_cols}")
    if n_buckets <= 0:
         raise ValueError("n_buckets must be a positive integer.")

    # Create a copy to avoid modifying the original data frame
    df_copy = data.copy()

    # 1. Quantile-based Binning: Returns integer bin indices (0 to n_buckets-1)
    df_copy['fico_bin_temp'] = pd.qcut(
        df_copy['fico_score'], q=n_buckets, labels=False, duplicates='drop'
    )

    # Adjust n_buckets if the requested number was not possible
    unique_bins = df_copy['fico_bin_temp'].nunique()
    if unique_bins < n_buckets:
        print(f"Warning: Only {unique_bins} buckets created due to data distribution.")
        n_buckets = unique_bins

    # 2. Define Rating Map: Map highest FICO bin (n_buckets-1) to Rating 1.
    rating_map = {i: n_buckets - i for i in range(n_buckets)}
    df_copy['credit_rating'] = df_copy['fico_bin_temp'].map(rating_map)

    # 3. Analyze and Format
    rating_stats = df_copy.groupby('credit_rating').agg(
        Min_FICO=('fico_score', 'min'),
        Max_FICO=('fico_score', 'max'),
        Count=('customer_id', 'count'),
        Default_Count=('default', 'sum'),
        Default_Rate=('default', 'mean')
    ).sort_values(by='credit_rating').reset_index()

    # Format the table for readability
    rating_stats['Default_Rate'] = (rating_stats['Default_Rate'] * 100).round(2).astype(str) + '%'
    rating_stats['FICO_Range'] = '[' + rating_stats['Min_FICO'].astype(int).astype(str) + ', ' + rating_stats['Max_FICO'].astype(int).astype(str) + ']'
    final_map = rating_stats[['credit_rating', 'FICO_Range', 'Count', 'Default_Count', 'Default_Rate']]

    return final_map

In [ ]:
# Load the data
try:
    df_loan_data = pd.read_csv("Task 3 and 4_Loan_Data.csv")
    print("Data loaded successfully.")
except FileNotFoundError:
    print("Error: 'Task 3 and 4_Loan_Data.csv' not found. Please ensure the file is uploaded.")
    exit()

# Define the desired number of buckets
N_BUCKETS = 5

# Generate the rating map (passing a copy of the data)
fico_rating_map = generate_fico_rating_map(df_loan_data.copy(), n_buckets=N_BUCKETS)

print(f"\n--- FICO Rating Map ({N_BUCKETS} Buckets, Quantile-based) ---")
print(fico_rating_map)

Data loaded successfully.

--- FICO Rating Map (5 Buckets, Quantile-based) ---
   credit_rating  FICO_Range  Count  Default_Count Default_Rate
0              1  [689, 850]   1993            108        5.42%
1              2  [654, 688]   1997            200       10.02%
2              3  [624, 653]   1989            301       15.13%
3              4  [588, 623]   1971            425       21.56%
4              5  [408, 587]   2050            817       39.85%
